<img src="https://full-stack-assets.s3.eu-west-3.amazonaws.com/M08-deep-learning/AT%26T_logo_2016.svg" alt="AT&T LOGO" width="50%" />

# Orange SPAM detector

## Project 🚧

One of the main pain point that AT&T users are facing is constant exposure to SPAM messages.

AT&T has been able to manually flag spam messages for a time, but they are looking for an automated way of detecting spams to protect their users.

## Goals 🎯

Your goal is to build a spam detector, that can automatically flag spams as they come based solely on the sms' content.

## Deliverable 📬

To complete this project, your team should: 

* Write a notebook that runs preprocessing and trains one or more deep learning models in order to predict the spam or ham nature of the sms
* State the achieved performance clearly

___

In [61]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import json

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [11]:
!python -m spacy download en_core_web_md -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [14]:
import en_core_web_md
nlp = en_core_web_md.load()

## EDA

In [16]:
df = pd.read_csv("../src/spam.csv", encoding="latin-1")

In [17]:
display(df.head())
display(df.describe(include="all"))

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [18]:
# missing values
df.isna().sum() / df.shape[0]

v1            0.000000
v2            0.000000
Unnamed: 2    0.991027
Unnamed: 3    0.997846
Unnamed: 4    0.998923
dtype: float64

In [19]:
# clean and rename columns
df["type"] = df["v1"]
df["text"] = df[df.columns[1:]].apply(lambda x: ",".join(x.dropna().astype(str)), axis=1)
df = df[["type", "text"]]

display(df.head())

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,"Ok lar... Joking wif u oni...,ham"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
px.histogram(df["type"])

## PRE PROCESSING

In [23]:
# encoding target
df["target"] = df["type"].map({"ham": 0,"spam": 1})

In [21]:
# cleaning text with nlp
df["text_clean"] = df["text"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" " or ch=="'"))
df["text_clean"] = df["text_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
df["text_clean"] = df["text_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))

df.head()

,type,text,text_clean
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,ham,"Ok lar... Joking wif u oni...,ham",ok lar joke wif u oniham
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c sayham
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live thoughham


In [27]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="out_of_vocab")
tokenizer.fit_on_texts(df["text_clean"])
df["text_encoded"] = tokenizer.texts_to_sequences(df["text_clean"])
df.head()

,type,text,text_clean,target,text_encoded
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,0,"[1, 262, 451, 535, 1, 29, 46, 205, 1, 75, 1, 9..."
1,ham,"Ok lar... Joking wif u oni...,ham",ok lar joke wif u oniham,0,"[16, 263, 620, 272, 2, 1]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1,"[12, 281, 3, 536, 730, 26, 1, 886, 422, 1, 1, ..."
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c sayham,0,"[2, 123, 173, 1, 2, 78, 1]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live thoughham,0,"[731, 20, 732, 133, 887]"


In [25]:
# backup pre preocessing
df.to_csv("preprocessing.csv", index=False)

In [28]:
text_pad = tf.keras.preprocessing.sequence.pad_sequences(df["text_encoded"], padding="post")
text_pad

array([[  1, 262, 451, ...,   0,   0,   0],
       [ 16, 263, 620, ...,   0,   0,   0],
       [ 12, 281,   3, ...,   0,   0,   0],
       ...,
       [  1, 986,   1, ...,   0,   0,   0],
       [112,   1,   1, ...,   0,   0,   0],
       [  1, 359,   1, ...,   0,   0,   0]])

In [31]:
X_train, X_val, Y_train, Y_val = train_test_split(text_pad, df["target"],
                                                  test_size=0.3,
                                                  random_state=0)

In [34]:
train = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
val = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

In [35]:
train_batch = train.shuffle(len(train)).batch(64)
val_batch = val.shuffle(len(val)).batch(64)

In [36]:
for text, typ in train_batch.take(1):  # checking batch content
  print(text, typ)

tf.Tensor(
[[  1 772 104 ...   0   0   0]
 [573  33 371 ...   0   0   0]
 [652   1  94 ...   0   0   0]
 ...
 [  1   1  20 ...   0   0   0]
 [  1 813   0 ...   0   0   0]
 [123  22 151 ...   0   0   0]], shape=(64, 74), dtype=int32) tf.Tensor(
[1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0], shape=(64,), dtype=int64)


## MODELING

### Baseline with LogisticRegression

In [46]:
lr = LogisticRegression(max_iter=1500)
lr.fit(X_train, Y_train)
Y_pred = lr.predict(X_val)

print(classification_report(Y_val, Y_pred))

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1434
           1       0.64      0.29      0.40       238

    accuracy                           0.88      1672
   macro avg       0.77      0.63      0.66      1672
weighted avg       0.86      0.88      0.85      1672



Not very useful but at least we have something

### Let's make our own deep learning model

In [54]:
vocab_size = tokenizer.num_words

model_simple = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, 64, input_shape=[text.shape[1]], name="embedding"),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_simple.summary()

c:\Users\giand\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 74, 64)         │        64,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,337 (274.75 KB)

 Trainable params: 70,337 (274.75 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.MeanSquaredError()
metric = tf.keras.metrics.MeanAbsoluteError()

model_simple.compile(optimizer=optimizer,
              loss=loss,
              metrics=[metric])

In [56]:
history = model_simple.fit(train_batch,
                           epochs=20,
                           validation_data=val_batch)

Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1314 - mean_absolute_error: 0.2605 - val_loss: 0.1156 - val_mean_absolute_error: 0.2213
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1054 - mean_absolute_error: 0.2249 - val_loss: 0.1115 - val_mean_absolute_error: 0.2238
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1004 - mean_absolute_error: 0.2187 - val_loss: 0.0977 - val_mean_absolute_error: 0.2315
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0824 - mean_absolute_error: 0.2063 - val_loss: 0.0763 - val_mean_absolute_error: 0.1654
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0622 - mean_absolute_error: 0.1503 - val_loss: 0.0637 - val_mean_absolute_error: 0.1690
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0458 - mean_absolute_error: 0.1228 - val_loss: 0.0489 - val_mean_absolute_error: 0.1168
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0338 - mean_absolute_error: 0.0938 - val_loss: 0.034

In [62]:
model_simple.save("model_simple.h5")
json.dump(model_simple.history.history, open("model_simple_history.json", "w"))

In [63]:
history = json.load(open("model_simple_history.json", "r"))

In [64]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history["loss"],
                         mode="lines",
                         name="loss"))
fig.add_trace(go.Scatter(y=history["val_loss"],
                         mode="lines",
                         name="val_loss"))
fig.show()